<a href="https://colab.research.google.com/github/matteo4diani/federated-survival-analysis/blob/main/federated_survival_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>